In [1]:
import pandas as pd
import sqlite3

# do not make a fucking dummies table
# you're not going to use it

In [2]:
# prepare sql database
conn = sqlite3.connect('siege.db')
cursor = conn.cursor()

In [3]:
def import_data(name, drop_last = False): # imports data into python, cleans it, and imports into sql

    if name == "everything":
        global everything
        
        everything = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\{name}.csv")
        everything = everything.drop(["Season", "Warm_Up"], axis = 1)
        
        # observations in Result column need to be standardized as convention changed over time
        results = list(everything["Result"])
        results_new = []
        for i in range(len(results)):
            results_new.append(results[i].lower())
        everything["Result"] = results_new

        everything.columns = everything.columns.str.lower()
        
        # import into sql
        cursor.execute("drop table if exists everything")
        everything.to_sql("everything", conn, index = False)
    
    else:
        global season
        
        season = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\{name}.csv")
        season = season.drop(["Date", "Session", "Warm_Up", "RP", "ΔRP"], axis = 1)
        
        if drop_last == True:
            season = season.drop(season.tail(1).index)

        season.columns = season.columns.str.lower()
        
        cursor.execute("drop table if exists season")
        season.to_sql("season", conn, index = False)
    return

In [4]:
import_data("everything")
import_data("y8s4_deep_freeze", drop_last = True) # change this depending on season & manual data cleanup

In [5]:
def get_sql_columns(name): # used for building the loop below
    cursor.execute(f"pragma table_info({name})")
    column_names = [desc[1] for desc in cursor.fetchall()]
    print(f"{name}: {column_names}")
    return

In [6]:
get_sql_columns("season")
get_sql_columns("everything")

season: ['game', 'map', 'squad_size', 'def_wins', 'def_losses', 'atk_wins', 'atk_losses', 'result', 'josh_score', 'josh_kills', 'josh_deaths', 'josh_assists', 'p1_score', 'p1_kills', 'p1_deaths', 'p1_assists', 'p2_score', 'p2_kills', 'p2_deaths', 'p2_assists', 'p3_score', 'p3_kills', 'p3_deaths', 'p3_assists', 'p4_score', 'p4_kills', 'p4_deaths', 'p4_assists', 'asa_score', 'asa_kills', 'asa_deaths', 'asa_assists', 'ed_score', 'ed_kills', 'ed_deaths', 'ed_assists', 'luke_score', 'luke_kills', 'luke_deaths', 'luke_assists']
everything: ['game', 'map', 'squad_size', 'def_wins', 'def_losses', 'atk_wins', 'atk_losses', 'result', 'josh_score', 'josh_kills', 'josh_deaths', 'josh_assists', 'asa_score', 'asa_kills', 'asa_deaths', 'asa_assists', 'ed_score', 'ed_kills', 'ed_deaths', 'ed_assists', 'luke_score', 'luke_kills', 'luke_deaths', 'luke_assists', 'p1_score', 'p1_kills', 'p1_deaths', 'p1_assists', 'p2_score', 'p2_kills', 'p2_deaths', 'p2_assists', 'p3_score', 'p3_kills', 'p3_deaths', 'p3_a

In [7]:
def get_player_data(player_name, dataframe): # gets individual player score, kills, deaths, assists, games and results

    cursor.execute(f"drop table if exists {player_name}_{dataframe}")
    cursor.execute(f"create table {player_name}_{dataframe} as \
                   select game, map, result, {player_name}_score, {player_name}_kills, {player_name}_deaths, {player_name}_assists from {dataframe}")
    cursor.execute(f"update {player_name}_{dataframe} set {player_name}_score = {player_name}_score - 2000 where result = 'win'")

    return

In [8]:
players = ["josh", "asa", "ed", "luke",
           "p1", "p2", "p3", "p4"]
dataframes = ["season", "everything"]

for p in players:
    for df in dataframes:
        get_player_data(p, df)

In [9]:
# pull the new tables back to python
player_tables = {f"{p}_{df}": pd.read_sql(f"select * from {p}_{df}", conn) for p in players for df in dataframes}

# shorthand, also leaves original dataframe in the dictionary in case i ruin these
josh_season, josh_everything = player_tables["josh_season"], player_tables["josh_everything"]
asa_season, asa_everything = player_tables["asa_season"], player_tables["asa_everything"]
ed_season, ed_everything = player_tables["ed_season"], player_tables["ed_everything"]
luke_season, luke_everything = player_tables["luke_season"], player_tables["luke_everything"]
p1_season, p1_everything = player_tables["p1_season"], player_tables["p1_everything"]
p2_season, p2_everything = player_tables["p2_season"], player_tables["p2_everything"]
p3_season, p3_everything = player_tables["p3_season"], player_tables["p3_everything"]
p4_season, p4_everything = player_tables["p4_season"], player_tables["p4_everything"]

In [10]:
# additional data cleanup

columns = [
                      "Josh_Score", "Josh_Kills", "Josh_Deaths", "Josh_Assists",
                      "Asa_Score", "Asa_Kills", "Asa_Deaths", "Asa_Assists",
                      "Ed_Score", "Ed_Kills", "Ed_Deaths", "Ed_Assists",
                      "Luke_Score", "Luke_Kills", "Luke_Deaths", "Luke_Assists",
                      "P1_Score", "P1_Kills", "P1_Deaths", "P1_Assists",
                      "P2_Score", "P2_Kills", "P2_Deaths", "P2_Assists",
                      "P3_Score", "P3_Kills", "P3_Deaths", "P3_Assists",
                      "P4_Score", "P4_Kills", "P4_Deaths", "P4_Assists"
]

columns = [c.lower() for c in columns]

season = season.drop(columns, axis = 1)
season = season.dropna()

everything = everything.drop(columns, axis = 1)
everything = everything.dropna()

# these columns have lots of empty entries that will ruin the tables if we dropna() before dropping these columns,
# and i don't need them anymore since they're in separate tables

In [11]:
# lists for all maps and squad sizes to iterate through
maps = sorted(list(everything["map"].unique()))
sizes = sorted(list(everything["squad_size"].unique()))

In [12]:
# overarching statistics for use in tables
season_wins = len(season[(season["result"] == "win")])
season_losses = len(season[(season["result"] == "loss")])
season_wl = season_wins/season_losses
season_games = len(season)

everything_wins = len(everything[(everything["result"] == "win")])
everything_losses = len(everything[(everything["result"] == "loss")])
everything_wl = everything_wins/everything_losses
everything_games = len(everything)

In [13]:
def win_loss_map(map_name):

    global season_map_wl, season_map_games, everything_map_wl, everything_map_games

    season_filtered = season[season["map"] == map_name]
    season_map_wins = len(season_filtered[(season_filtered["result"] == "win")])
    season_map_losses = len(season_filtered[(season_filtered["result"] == "loss")])
    season_map_games = len(season_filtered)
    if season_map_losses == 0:
        season_map_wl = season_map_wins
    else:
        season_map_wl = season_map_wins/season_map_losses

    everything_filtered = everything[everything["map"] == map_name]
    everything_map_wins = len(everything_filtered[(everything_filtered["result"] == "win")])
    everything_map_losses = len(everything_filtered[(everything_filtered["result"] == "loss")])
    everything_map_games = len(everything_filtered)
    if everything_map_losses == 0:
        everything_map_wl = everything_map_wins
    else:
        everything_map_wl = everything_map_wins/everything_map_losses

    return season_map_wl, season_map_games, everything_map_wl, everything_map_games

In [14]:
def win_loss_squad(squad_size):

    global season_squad_wl, season_squad_games, everything_squad_wl, everything_squad_games

    season_filtered = season[season["squad_size"] == squad_size]
    season_squad_wins = len(season_filtered[(season_filtered["result"] == "win")])
    season_squad_losses = len(season_filtered[(season_filtered["result"] == "loss")])
    season_squad_games = len(season_filtered)

    if season_squad_losses == 0:
        season_squad_wl = season_squad_wins
    else:
        season_squad_wl = season_squad_wins/season_squad_losses

    everything_filtered = everything[everything["squad_size"] == squad_size]
    everything_squad_wins = len(everything_filtered[(everything_filtered["result"] == "win")])
    everything_squad_losses = len(everything_filtered[(everything_filtered["result"] == "loss")])
    everything_squad_games = len(everything_filtered)

    if everything_squad_losses == 0:
        everything_squad_wl = everything_squad_wins
    else:
        everything_squad_wl = everything_squad_wins/everything_squad_losses

    return season_squad_wl, season_squad_games, everything_squad_wl, everything_squad_games

In [15]:
# building a win/loss table for maps

wl_maps = ["Average W/L"]
season_wl_ratios = [season_wl]
season_wl_count = ["na"]
everything_wl_ratios = [everything_wl]
everything_wl_count = ["na"]
# initial inputs set the first row of my final tables

for m in maps:
    win_loss_map(m)
    wl_maps.append(m)
    season_wl_ratios.append(season_map_wl)
    season_wl_count.append(season_map_games)
    everything_wl_ratios.append(everything_map_wl)
    everything_wl_count.append(everything_map_games)

wl_maps_table = pd.DataFrame({"map": wl_maps,
                              "season_wl": season_wl_ratios,
                              "season_games": season_wl_count,
                              "everything_wl": everything_wl_ratios,
                              "everything_games": everything_wl_count})
wl_maps_table.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\wl_maps_y8s4.csv")

In [16]:
# building a win/loss table for squads

wl_squads = ["Average W/L"]
season_wl_ratios = [season_wl]
season_wl_count = ["na"]
everything_wl_ratios = [everything_wl]
everything_wl_count = ["na"]
# initial inputs set the first row of my final tables

for s in sizes:
    win_loss_squad(s)
    wl_squads.append(s)
    season_wl_ratios.append(season_squad_wl)
    season_wl_count.append(season_squad_games)
    everything_wl_ratios.append(everything_squad_wl)
    everything_wl_count.append(everything_squad_games)

wl_squads_table = pd.DataFrame({"size": wl_squads,
                                "season_wl": season_wl_ratios,
                                "season_games": season_wl_count,
                                "everything_wl": everything_wl_ratios,
                                "everything_games": everything_wl_count})
wl_squads_table.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\wl_squads_y8s4.csv")

In [17]:
def performance(p, df): # something is going wrong, fi xit
    
    global avg_score, kd, wl

    d = player_tables[f"{p}_{df}"]
    d = d.dropna()
    
    total_score = d[f"{p}_score"].sum()
    games = len(d)
    
    kills = d[f"{p}_kills"].sum()
    deaths = d[f"{p}_deaths"].sum()
    
    wins = len(d[(d["result"] == "win")])
    losses = len(d[(d["result"] == "loss")])

    avg_score = total_score / games if games > 0 else total_score
    kd = kills / deaths if deaths > 0 else kills
    wl = wins / losses if losses > 0 else wins

    return avg_score, kd, wl

In [18]:
perform = pd.DataFrame()
columns = [f"{p}_{df}" for p in players for df in dataframes]

scores = []
kdr = []
wlr = []

for p in players:
    for df in dataframes:
        performance(p, df)
        scores.append(avg_score)
        kdr.append(kd)
        wlr.append(wl)

perform["Player & Timeframe"] = columns
perform["Score"] = scores
perform["W/L"] = wlr
perform["K/D"] = kdr

perform.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\performance_summary.csv")